<h1>Steam Gaming Data Engineering<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

### by Sebastian Wolf

### Tasks:

- Install and run PySpark.
- Load .csv for `Player_Summaries`, `Game_Publishers`, `Game_Genres`, `Game_Developers`, `Games_1` into PySpark dataframes.
- Join all `Games_` tables into one dataframe.
- Count the number of games per `publisher` and per `genre`.
- Find day and hour when most new accounts were created (based on Player_Summaries table) e.g. 8pm on 14th August 2005.

# Setup

In [1]:
# Make sure script changes take effect within this session
%load_ext autoreload
%autoreload 2

In [2]:
# import some useful packages for this analysis, start spark session
from setup import *

In [3]:
spark

# Load csvs

In [4]:
# list of tables we want to load
dfs = ['Player_Summaries', 'Games_Publishers', 'Games_Genres', 'Games_Developers', 'Games_1']

In [5]:
# load raw data and count rows
spark_handler = spark_df_handler()
print('Number of records \n')
for each in dfs:
    spark_handler.load(each)
    print('{0} : {1}'.format(each, str(spark_handler.dfraw[each].count())))

Number of records 
 

Player_Summaries : 5000000
Games_Publishers : 18761
Games_Genres : 39669
Games_Developers : 19333
Games_1 : 15426029


In [6]:
# check player summary table
spark_handler.dfraw[dfs[0]].limit(5).toPandas()

,steamid,personaname,profileurl,avatar,avatarmedium,avatarfull,personastate,communityvisibilitystate,profilestate,lastlogoff,commentpermission,realname,primaryclanid,timecreated,gameid,gameserverip,gameextrainfo,cityid,loccountrycode,locstatecode,loccityid,dateretrieved
0,76561197996959226,Uragan,http://steamcommunity.com/profiles/76561197996...,http://media.steampowered.com/steamcommunity/p...,http://media.steampowered.com/steamcommunity/p...,http://media.steampowered.com/steamcommunity/p...,0,3,1,2012-08-06 17:28:40,None,Sergey,103582791430157457,2008-02-29 07:43:57,None,None,None,None,RU,48,41548,2013-03-03 18:00:04
1,76561198004086494,matuwer,http://steamcommunity.com/profiles/76561198004...,http://media.steampowered.com/steamcommunity/p...,http://media.steampowered.com/steamcommunity/p...,http://media.steampowered.com/steamcommunity/p...,0,3,1,2009-12-05 04:50:41,None,None,103582791429524891,2008-12-17 12:10:20,None,None,None,None,None,None,None,2013-03-04 21:35:56
2,76561198039210349,mrgameking,http://steamcommunity.com/id/mrgameking/,http://media.steampowered.com/steamcommunity/p...,http://media.steampowered.com/steamcommunity/p...,http://media.steampowered.com/steamcommunity/p...,1,3,1,2013-03-10 12:45:16,None,None,103582791433803066,2011-03-11 08:26:28,None,None,None,None,NL,11,None,2013-03-11 14:52:33
3,76561198059144452,ACÏŸDC,http://steamcommunity.com/profiles/76561198059...,http://media.steampowered.com/steamcommunity/p...,http://media.steampowered.com/steamcommunity/p...,http://media.steampowered.com/steamcommunity/p...,0,3,1,2013-03-11 14:45:54,None,Ð¡ÐµÑ€Ð³ÐµÐ¹,103582791432715487,2012-02-20 05:53:25,None,None,None,None,BY,03,None,2013-03-12 02:51:17
4,76561198074500506,Rayman,http://steamcommunity.com/id/Rayman99971/,http://media.steampowered.com/steamcommunity/p...,http://media.steampowered.com/steamcommunity/p...,http://media.steampowered.com/steamcommunity/p...,0,3,1,2013-03-13 02:02:39,1,Cory,103582791433073276,2012-10-27 22:16:34,None,None,None,None,US,OR,3187,2013-03-14 03:25:26


In [7]:
# check publisher table
spark_handler.dfraw[dfs[1]].limit(5).toPandas()

,appid,Publisher
0,207990,None
1,215220,None
2,220824,None
3,241600,None
4,249310,None


In [8]:
# check genres table
spark_handler.dfraw[dfs[2]].limit(5).toPandas()

,appid,Genre
0,7290,RPG
1,8980,RPG
2,18010,RPG
3,18040,RPG
4,24447,RPG


In [9]:
# check developers table
spark_handler.dfraw[dfs[3]].limit(5).toPandas()

,appid,Developer
0,462530,8i
1,452420,M2
2,466530,M2
3,366960,UD
4,315060,2SD


In [10]:
# check games table
spark_handler.dfraw[dfs[4]].limit(5).toPandas()

,steamid,appid,playtime_2weeks,playtime_forever,dateretrieved
0,76561197972368092,55230,None,2919,2013-05-13 06:11:48 UTC
1,76561197972228702,42910,None,18,2013-05-13 03:14:44 UTC
2,76561198058121020,33900,None,13,2013-08-22 12:01:04 UTC
3,76561198005218214,34330,None,349,2013-06-14 09:36:15 UTC
4,76561198005175274,1250,None,547,2013-06-14 08:10:50 UTC


# Join all `Games_` tables into one dataframe

In [11]:
# join the games tables
games = spark_handler.dfraw[dfs[1]]\
    .join(spark_handler.dfraw[dfs[2]], 'appid')\
    .join(spark_handler.dfraw[dfs[3]], 'appid')\
    .join(spark_handler.dfraw[dfs[4]], 'appid')
games.limit(5).toPandas()

,appid,Publisher,Genre,Developer,steamid,playtime_2weeks,playtime_forever,dateretrieved
0,108800,Electronic Arts,Action,Crytek Studios,76561197973718054,None,1348,2013-05-14 07:52:54 UTC
1,108800,Electronic Arts,Action,Crytek Studios,76561198077027953,213,969,2013-09-27 11:45:42 UTC
2,108800,Electronic Arts,Action,Crytek Studios,76561197970336122,None,998,2013-05-11 20:12:30 UTC
3,108800,Electronic Arts,Action,Crytek Studios,76561198050351845,102,102,2013-08-28 07:52:55 UTC
4,108800,Electronic Arts,Action,Crytek Studios,76561197983834231,None,942,2013-05-18 15:08:30 UTC


# Count the number of games per `publisher` and per `genre`

In [12]:
# count games per publisher
games_per_publisher = games.groupby('Publisher').agg(F.countDistinct('appid').alias('number_of_games')).toPandas()
games_per_publisher

,Publisher,number_of_games
0,Iceberg Interactive,11
1,11 bit studios,2
2,Carpe Fulgur LLC,3
3,Team17 Digital Ltd,13
4,Rocket Bear Games,1
...,...,...
643,Kabam,1
644,Cardboard Computer,1
645,Futuremark,3
646,Secret Exit Ltd.,1


In [13]:
# count games per genre
games_per_genre = games.groupby('Genre').agg(F.countDistinct('appid').alias('number_of_games')).toPandas()
games_per_genre

,Genre,number_of_games
0,Education,3
1,Massively Multiplayer,43
2,Adventure,455
3,Sports,56
4,Accounting,1
5,Audio Production,1
6,Video Production,2
7,Animation & Modeling,6
8,Racing,102
9,Design & Illustration,6


# Find day and hour when most new accounts were created (based on Player_Summaries table) e.g. 8pm on 14th August 2005

In [14]:
# get the summary table
player_summaries = spark_handler.dfraw[dfs[0]]

In [15]:
# Create timestamp vars
player_summaries = player_summaries.withColumn('datetimecreated', F.to_timestamp(player_summaries.timecreated,'yyyy-MM-dd HH:mm:ss'))
player_summaries = player_summaries.withColumn('day_created', F.date_trunc('day', player_summaries['datetimecreated']))
player_summaries = player_summaries.withColumn('dayhour_created', F.date_trunc('hour', player_summaries['datetimecreated']))
player_summaries = player_summaries.withColumn('hour_created', F.hour('datetimecreated').cast('byte'))

In [16]:
# make sure players appear only once
player_summaries.count() == player_summaries.select('steamid').distinct().count()

True

In [17]:
# get account creation count by dayhour
player_summaries_dayhour = player_summaries.groupby('dayhour_created').count().orderBy('count', ascending = False)
player_summaries_dayhour.limit(10).toPandas()

,dayhour_created,count
0,NaT,252717
1,2013-03-02 10:00:00,1782
2,2012-12-25 10:00:00,1448
3,2012-12-25 08:00:00,1355
4,2012-12-25 11:00:00,1159
5,2012-12-25 07:00:00,1086
6,2012-11-28 09:00:00,1079
7,2004-11-16 17:00:00,1056
8,2012-11-28 10:00:00,1049
9,2012-12-25 12:00:00,1046


In [18]:
# Just make sure we didn't lose dates due to reformatting of timestamps
player_summaries.select([F.count(F.when(F.isnan('timecreated') | F.col('timecreated').isNull() , True))]).toPandas()

,count(CASE WHEN (isnan(timecreated) OR (timecreated IS NULL)) THEN true END)
0,252694


In [19]:
# get account creation count by day
player_summaries_day = player_summaries.groupby('day_created').count().orderBy('count', ascending = False)
player_summaries_day.limit(10).toPandas()

,day_created,count
0,NaT,252717
1,2012-12-25,18511
2,2012-11-28,16356
3,2013-03-02,13510
4,2012-12-26,13392
5,2012-09-15,13344
6,2013-02-23,13316
7,2012-12-24,13101
8,2012-09-22,12892
9,2012-08-18,12610


In [20]:
# get account creation count by hour
player_summaries_hour = player_summaries.where(F.col('day_created') == dt.datetime(2012,12,25)).groupby('hour_created').count().orderBy('count', ascending = False)
player_summaries_hour.limit(24).toPandas()

,hour_created,count
0,10,1448
1,8,1355
2,11,1159
3,7,1086
4,12,1046
5,6,1040
6,3,968
7,13,962
8,5,957
9,4,951


## Answer:
- the single hour when most new accounts were created was 10am on 2nd March 2013
- the single day when most new accounts were created was the 25th of December 2012
- on the 25th of December 2012, the single day when most new accounts were created, the hour when most accounts were created was 10am